In [20]:
import sympy as sp
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting

init_vprinting(use_latex='mathjax')

#Defining rotation matrix builder
def getRot(axis, var):
    if axis == '1':
        return sp.Matrix([[1, 0, 0],
                [0, sp.cos(var), -sp.sin(var)],
                [0,sp.sin(var), sp.cos(var)]])
    if axis == '2':
        return sp.Matrix([[sp.cos(var), 0, sp.sin(var)],
                [0, 1, 0],
                [-sp.sin(var), 0, sp.cos(var)]])
    if axis == '3':
        return sp.Matrix([[sp.cos(var), -sp.sin(var), 0],
               [sp.sin(var), sp.cos(var), 0],
               [0, 0, 1]])
    else:
        print('Invalid entry')
        
#Function that simplifies elements in a matrix
def simplifyMatrix(matrix):
    for i in range(len(matrix.col(0))):
        for j in range(len(matrix.row(0))):
            matrix[i,j] = matrix[i,j].simplify()
    return matrix

#Cross product
def cross(v1,v2):
    icomp  = v1[1]*v2[2]-v1[2]*v2[1]
    jcomp = v1[2]*v2[0]-v1[0]*v2[2]
    kcomp = v1[0]*v2[1]-v1[1]*v2[0]
    return sp.Matrix([icomp,jcomp,kcomp])


#Functions that creates skew symettric matrix from vector
def skew(a):
    return sp.Matrix([
        [0, -a[2], a[1]],
        [a[2], 0, -a[0]],
        [-a[1], a[0], 0]])

#Functions that creates vector from skew symettric matrix
def unskew(a):
    if ((a[2,1] == -a[1,2]) and (a[0,2]== -a[2,0]) and (a[1,0] == -a[0,1])):
        return sp.Matrix([
            [a[2,1]],
            [a[0,2]],
            [a[1,0]]])
    else:
        print('Matrix is not skew symmetric')
        return


def principalJ(J1,J2,J3):
    return sp.Matrix([[J1,0,0],
                      [0,J2,0],
                      [0,0,J3]])

def degToRad(x):
    return x*sp.pi/180

def radToDeg(x):
    return x*180/sp.pi

def displayH(H):
    s = sp.Function('s')
    c = sp.Function('c')
    display(H.replace(sp.sin,s).replace(sp.cos,c))

In [42]:
#defining variables:
t = sp.symbols('t')
theta = sp.Function('theta')(t)
phi = sp.Function('phi')(t)
psi = sp.Function('psi')(t)

#rotation matrices:
R0B = getRot('3',psi)*getRot('2',theta)*getRot('1',phi)
RB0 = R0B.T

#angular velocity of 3 frame wrt 0 frame, expressed in the 0 frame
OmegaB0 = simplifyMatrix(R0B.T*R0B.diff(t))#skewed 
omegaB0 = unskew(OmegaB0)# unskewed

print('R0B:')
displayH(simplifyMatrix(R0B))
print('omegaB0, skewe:')
displayH(OmegaB0)
print('omegaB0_B,unskewed:')
displayH(omegaB0)

eulerRates = sp.Matrix([phi.diff(t),theta.diff(t),psi.diff(t)]) #roll, pitch, yaw


R0B:


⎡c(ψ)⋅c(θ)  -c(φ)⋅s(ψ) + c(ψ)⋅s(φ)⋅s(θ)  c(φ)⋅c(ψ)⋅s(θ) + s(φ)⋅s(ψ)⎤
⎢                                                                  ⎥
⎢c(θ)⋅s(ψ)  c(φ)⋅c(ψ) + s(φ)⋅s(ψ)⋅s(θ)   c(φ)⋅s(ψ)⋅s(θ) - c(ψ)⋅s(φ)⎥
⎢                                                                  ⎥
⎣  -s(θ)             c(θ)⋅s(φ)                   c(φ)⋅c(θ)         ⎦

omegaB0, skewe:


⎡          0            -c(φ)⋅c(θ)⋅ψ̇ + s(φ)⋅θ̇  c(φ)⋅θ̇ + c(θ)⋅s(φ)⋅ψ̇⎤
⎢                                                                  ⎥
⎢c(φ)⋅c(θ)⋅ψ̇ - s(φ)⋅θ̇             0                 s(θ)⋅ψ̇ - φ̇     ⎥
⎢                                                                  ⎥
⎣-c(φ)⋅θ̇ - c(θ)⋅s(φ)⋅ψ̇       -s(θ)⋅ψ̇ + φ̇                0          ⎦

omegaB0_B,unskewed:


⎡    -s(θ)⋅ψ̇ + φ̇     ⎤
⎢                    ⎥
⎢c(φ)⋅θ̇ + c(θ)⋅s(φ)⋅ψ̇⎥
⎢                    ⎥
⎣c(φ)⋅c(θ)⋅ψ̇ - s(φ)⋅θ̇⎦

In [41]:
A = sp.zeros(3,3)
for i in range(3):
    for j in range(3):
        A[i,j]+= (omegaB0[i].coeff(eulerRates[j]))

display(simplifyMatrix(A.inv()))

⎡1  sin(φ)⋅tan(θ)  cos(φ)⋅tan(θ)⎤
⎢                               ⎥
⎢0     cos(φ)         -sin(φ)   ⎥
⎢                               ⎥
⎢      sin(φ)         cos(φ)    ⎥
⎢0     ──────         ──────    ⎥
⎣      cos(θ)         cos(θ)    ⎦